In [2]:
import numpy as np
import pandas as pd
import json
from scipy.signal import find_peaks
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

import scanpy as sc
import squidpy as sq

import src.utilities as utils
from src.slide_utilities import *

import warnings
warnings.filterwarnings('ignore')

In [3]:
adata_main = sc.read_h5ad("../03_image_registration/data_level_1/adata_slide2_with_Ki67_noX8_v1.h5ad")
del adata_main.layers['counts']

Add domain generated by ```01_nichepca_per_sample.ipynb``` to ```adata_main```

In [7]:
adata_main.obs['sample'] = [i.upper().split('_')[0] + "_slide2" for i in adata_main.obs['sample']]
sample_set = adata_main.obs['sample'].unique()
sample_set

array(['X1_slide2', 'X2_slide2', 'X3_slide2', 'X4_slide2', 'X5_slide2',
       'X6_slide2', 'X7_slide2'], dtype=object)

In [8]:
resolutions = np.arange(0.03, 0.4, 0.02)
resolutions = resolutions.round(4).tolist()
for res in resolutions:
    adata_main.obs[f'per_sample_leiden_res_{res}'] = '0'

In [12]:
data_path = "/projectbig/jupyternotebook/xenium_cgn/domain_identification_from_epyc/per_sample_dist/"

for sample in tqdm(sample_set):
    adata_per_sample = sc.read_h5ad(data_path + f"adata_nichepca_sample_{sample.lower().split('_')[0]}_clustered_slide2_with_Ki67.h5ad")
    
    for res in resolutions:
        adata_main.obs.loc[adata_main.obs['sample'] == sample ,f'per_sample_leiden_res_{res}'] = adata_per_sample.obs[f'leiden_res_{res}']

    # if sample in ['X60', 'X25', 'X7', 'X44', 'X10']:
    #     adata_main.write("data/adata_nichepca_slide_clustered_per_sample_tmp.h5ad")

100%|██████████| 7/7 [00:03<00:00,  2.19it/s]


In [13]:
adata_main.obs

,x,y,z,cluster,n_transcripts,density,elongation,area,avg_confidence,avg_assignment_confidence,...,per_sample_leiden_res_0.21,per_sample_leiden_res_0.23,per_sample_leiden_res_0.25,per_sample_leiden_res_0.27,per_sample_leiden_res_0.29,per_sample_leiden_res_0.31,per_sample_leiden_res_0.33,per_sample_leiden_res_0.35,per_sample_leiden_res_0.37,per_sample_leiden_res_0.39
cell_CR9fbacb852-172697-0-0-0-0-0-0,5686.630300,16243.587750,32.710325,4,8,0.3695,3.021,21.650,0.9724,0.9025,...,2,2,2,2,2,2,2,2,1,2
cell_CR9fbacb852-183840-0-0-0-0-0-0,5533.867400,17797.181667,28.890549,3,6,0.8171,4.300,7.343,0.9859,0.9700,...,3,3,3,3,3,3,3,3,3,3
cell_CR9fbacb852-183965-0-0-0-0-0-0,5544.457950,17818.821750,27.178570,3,8,2.2230,2.835,3.599,0.9743,0.9775,...,3,3,3,3,3,3,3,3,3,3
cell_CR9fbacb852-184398-0-0-0-0-0-0,7113.689200,16393.493100,23.298814,4,10,1.3270,1.694,7.536,0.9939,0.9780,...,2,2,2,2,2,2,2,2,1,2
cell_CR9fbacb852-185040-0-0-0-0-0-0,7705.488675,16117.277875,23.637515,4,12,1.7790,1.445,6.745,0.9983,0.9950,...,1,1,1,1,1,1,1,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cell_CR9fbacb852-344135-1,7156.741758,2170.739595,38.006215,1,19,0.9734,2.325,19.520,0.9967,0.7937,...,0,0,0,0,0,0,0,0,0,0
cell_CR9fbacb852-344237-1,7246.011350,3333.079183,34.770267,4,18,1.0500,25.290,17.140,0.9914,0.6889,...,2,2,2,2,1,1,1,1,1,1
cell_CR9fbacb852-344280-1,7478.337415,2462.420563,34.261774,4,41,1.6890,9.127,24.270,0.9982,0.7439,...,4,4,4,4,4,4,4,4,3,3
cell_CR9fbacb852-344416-1,7380.969613,2642.821607,38.622404,4,15,2.0910,4.592,7.173,0.9983,0.7467,...,4,4,4,4,4,4,4,4,3,3


In [14]:
adata_main.write("data/adata_nichepca_slide2_clustered_per_sample_tmp.h5ad")
"data/adata_nichepca_slide2_clustered_per_sample_tmp.h5ad"

'data/adata_nichepca_slide2_clustered_per_sample_tmp.h5ad'